# Create models from *Ergo* data
This notebook contains the code for models used to predict the *Ergo* data. See the report [here](https://git.cs.sun.ac.za/Computer-Science/rw771/2022/26723077-TG7-doc) or the source code behind the data [here](https://git.cs.sun.ac.za/Computer-Science/rw771/2022/26723077-TG7-src).

### Items to explore

- What is the optimal number of PCs for model performance?
- Which is the optimal dimensionality reduction method: tSNE, Autoencoder, PCA?
- Compare different model types: Random Forest, SVM, NN, Nïeve Bayes, Quadratic Discriminent Analysis

## Imports and constants

In [ ]:
%load_ext autoreload
%autoreload 2

from common_utils import *

import pickle
from matplotlib import cm
from time import time

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Models
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Distributions
from sklearn.utils.fixes import loguniform

In [ ]:
USE_W_AND_B = False

## Preprocessing data

In [ ]:
# Get some metadata and descriptions about the gestures
gesture_info = get_gesture_info()

# Get a hashmap of the form `directory` => [`file1`, `file2`, ...]
dir_files = get_dir_files()

# Print out all the gestures that we've got data for
format_string = "\n- ".join([
    f'{k}: {gesture_info.get(k, {}).get("description", "<No description>"):<40} ({len(v)} files)' 
    for k,v in dir_files.items()
])
print(f'The following gestures have data recorded for them:\n- {format_string}')

## Read in the data to an `np.array`

TODO: Convert this into a method in `common_utils.py`

In [ ]:
n_sensors, n_timesteps = 30, 40

# Exclude all classes which do not have at least 300 observations
n_classes = len([d for d,fs in dir_files.items() if len(fs) > 300])
n_obs = sum([len(fs) for d, fs in dir_files.items() if len(fs) > 300])
print(f'{n_classes=}, {n_obs=}')

# Create arrays to store the observations and labels
X = np.zeros((n_obs, n_sensors * n_timesteps))
y = np.zeros((n_obs,))
# Also keep track of the paths from which each observation originated
paths = []

# Create hashmaps to easily convert from and from indexes (0, 1, 2, 3, ...) 
# and gestures ('gesture0001', 'gesture0002', ...)
idx_to_gesture = {}
gesture_to_idx = {}

# The `obs_idx` increments with every observation
obs_idx = 0
# The `label_idx` increments with every label iff 
# there's > 300 observations for that label
label_idx = 0

# Iterate over every gesture
for gesture_index, filenames in dir_files.items():
    if len(filenames) > 300:
        # Populate the idx <-> gesture mapping
        idx_to_gesture[label_idx] = gesture_index
        gesture_to_idx[gesture_index] = label_idx
        
        
        print(f"Processing {gesture_index} ({gesture_info[gesture_index]['desc']})")
        # Iterate over every observation for the current gesture
        for file in filenames:
            # Keep track of the path for bookkeeping purposes
            paths.append(f'../gesture_data/train/{gesture_index}/{file}')
            # Read in the raw sensor data. Normalisation is done later on via sklearn
            df = read_to_df(paths[-1], normalise=False)
            # Make sure the data is the correct shape
            if df.shape != (n_timesteps, n_sensors):
                print(df)
            # Save observation as a plain `np.array` in the X array
            X[obs_idx] = df.to_numpy().T.flatten()
            # If there are any NaNs, the observation is probably 
            # incomplete and should be removed
            if np.any(np.isnan(X[obs_idx])):
                print(f'rm {paths[-1]}')
            # Save the label of the current observation
            y[obs_idx] = label_idx
            
            obs_idx += 1
        label_idx += 1
        
print('done')

### Train-test split and scale the data

In [ ]:
X_train, X_test, y_train, y_test, paths_train, paths_test = train_test_split(
    X, y, np.array(paths), test_size=0.25, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Train models without dimensionality reduction


Training KNeighborsClassifier(n_neighbors=21)
- Time taken: 153.927s
- Best performing model
`KNeighborsClassifier(algorithm='ball_tree', n_neighbors=21)`
- Score: train: 0.9679, test: 0.9774


Training MLPClassifier(max_iter=1000)
- Time taken: 81.279s
- Best performing model
```MLPClassifier(activation='tanh', alpha=0.0013330009770265291, hidden_layer_sizes=400, max_iter=1000)```
- Score: train: 0.9907, test: 0.9947

Training MLPClassifier(max_iter=1000)
- Time taken: 27.189s
- Best performing model
`MLPClassifier(alpha=0.0016877545702567223, hidden_layer_sizes=100, max_iter=1000)`
- Score: train: 0.9903, test: 0.9950


15h44

In [ ]:
%%time
models = []

models.append(
    (MLPClassifier(max_iter=1000), {
        'hidden_layer_sizes': [(100), (200), (400), (100, 50), (200, 100), (400, 200), 
                               (100, 50, 25), (200, 100, 50), (400, 200, 100)],
#         'hidden_layer_sizes': [(50)],
        'activation' : ['logistic', 'tanh', 'relu'],
        'solver' : ['lbfgs', 'adam'],
        'alpha': loguniform(1e-6, 1e-2),
    })
)

clfs = []
for model, param_grid in models:
    print(f'\nTraining {model}')
    start = time()
    clf = RandomizedSearchCV(
        model, param_grid, n_iter=10
    )
    clf = clf.fit(X_train, y_train)
    print(f'- Time taken: {time() - start:.3f}s\n- Best performing model\n`{clf.best_estimator_}`\n- Score: train: {clf.best_score_:.4f}, test: {clf.score(X_test, y_test):.4f}')
    clfs.append(clf.best_estimator_)
    

### Visualise important features from full-dimensionality classifiers

In [ ]:
mlp = clfs[0]

In [ ]:
num_cols = 4
fig, axes = plt.subplots(n_classes//num_cols+1, num_cols)
# use global min / max to ensure all weights are shown on the same scale

vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for gesture_idx, ax in enumerate(axes.ravel()):
    if gesture_idx >= n_classes:
        ax.set_xticks(())
        ax.set_yticks(())
        ax.grid(False)
        continue
        
    multiplied = mlp.coefs_[0]
    for layer in range(1, len(mlp.coefs_)):
        multiplied = multiplied @ mlp.coefs_[layer]

    importances = multiplied[:, gesture_idx].reshape(n_sensors, n_timesteps)
#     importances = (mlp.coefs_[0] @ mlp.coefs_[1][:, gesture_idx]).reshape(n_sensors, n_timesteps)
    
    gesture_label = idx_to_gesture[gesture_idx]
    gesture_description = gesture_info[gesture_label]['description']
    plot_raw_gesture(
        importances.reshape(n_sensors, n_timesteps).T,
        f'{gesture_label}\n{gesture_description}',
        ax=ax,
        show_cbar=False,
        show_xticks=False,
        show_yticks=False,
        delim_lw=0
    )

plt.suptitle('Importances per gesture for the trained MLP')
plt.tight_layout()
plt.savefig('imgs/importances.pdf')

In [ ]:

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

fig, axes = plt.subplots(5, 5)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(
        coef.reshape(n_sensors, n_timesteps), 
        cmap=plt.cm.gray, 
        vmin=0.5 * vmin, 
        vmax=0.5 * vmax
    )
    ax.set_xticks(())
    ax.set_yticks(())

### Reduce dimensionality via PCA

In [ ]:
%%time
n_components = 10

print(f'Fitting PCA with {n_components} components on {X_train.shape[0]} observations')
pca = PCA(
    n_components=n_components,
    svd_solver="randomized", 
    whiten=True
).fit(X_train)

print(f"PCA explained {100*sum(pca.explained_variance_ratio_):.2f}% of the variance with {n_components} PCs")

# PCA-transform the input test and train data
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

## Train multiple models on the data
Each model is defined in its own cell, and appended to the list `models`. 
After all the definitions, every model in the list is trained and evaluated.

In [ ]:
models = []

---
Support Vector Machine
- Time taken: 7.731s
- Best performing model
`SVC(C=35891.14381335473, class_weight='balanced', gamma=0.057667711437645666)`
- Score: train: 0.9896, test: 0.9901

In [ ]:
models.append(
    (SVC(kernel="rbf", class_weight="balanced"), {
        "C": loguniform(1e3, 1e5),
        "gamma": loguniform(1e-4, 1e-1),
    })
)

---
Linear Support Vector Machine

- Time taken: 367.828s
- Best performing model: SVC(C=1284.851851420933, class_weight='balanced', kernel='linear')
- Score: train: 0.9780, test: 0.9784

In [ ]:
models.append(
    (SVC(kernel="linear", class_weight="balanced"), {
        "C": loguniform(1e3, 1e5),
    })
)

---
K-Nearest Neighbours
- Time taken: 0.820s
- Best performing model
`KNeighborsClassifier(algorithm='ball_tree', n_neighbors=21)`
- Score: train: 0.9849, test: 0.9851

In [ ]:
models.append(
    (KNeighborsClassifier(n_neighbors=n_classes), {
        "algorithm": ['ball_tree', 'kd_tree', 'brute'],
    })
)

---
Ada Boost
- Time taken: 268.507s
- Best performing model: AdaBoostClassifier(learning_rate=0.20109497487437183, n_estimators=792)
- Score: train: 0.7856, test: 0.7867


In [ ]:
models.append(
    (AdaBoostClassifier(), {
        'n_estimators': range(10, 1000),
        'learning_rate': np.linspace(1e-4, 2, num=200),
    })
)

---
Decision Tree
- Time taken: 1.136s
- Best performing model
`DecisionTreeClassifier(class_weight='balanced', max_depth=186,
                       min_samples_split=0.26530612244897955)`
- Score: train: 0.4396, test: 0.4712

In [ ]:
models.append(
    (DecisionTreeClassifier(class_weight="balanced"), {
        'max_depth': range(1, 200),
        'min_samples_split': np.linspace(0, 1),
    })
)

---
Random Forest
- Time taken: 31.299s
- Best performing model

`RandomForestClassifier(max_depth=187, max_features=0.836734693877551,
                       min_samples_split=0.16326530612244897, n_estimators=176)`
- Score: train: 0.6110, test: 0.6095

In [ ]:
models.append(
    (RandomForestClassifier(), {
        'n_estimators': range(10, 500), 
        'max_depth': range(1, 200), 
        'max_features': np.linspace(0, 1),
        'min_samples_split': np.linspace(0, 1),
    })
)

---

Multi-layer Perceptron
- Time taken: 484.457s
- Best performing model

```MLPClassifier(activation='tanh', alpha=2.8761865563186644e-05,
              hidden_layer_sizes=(400, 200), max_iter=1000)```
- Score: train: 0.9914, test: 0.9919

In [ ]:
models.append(
    (MLPClassifier(max_iter=1000), {
        'hidden_layer_sizes': [(100), (200), (400), (100, 50), (200, 100), (400, 200), 
                               (100, 50, 25), (200, 100, 50), (400, 200, 100)],
        'activation' : ['logistic', 'tanh', 'relu'],
        'solver' : ['lbfgs', 'adam'],
        'alpha': loguniform(1e-6, 1e-2),
    })
)

---
Gaussian Naïve Bayes
- Time taken: 0.031s
- Best performing model: GaussianNB()
- Score: train: 0.9255, test: 0.9264

In [ ]:
models.append((GaussianNB(), {}))

---
Quadratic Discriminant Analysis
- Time taken: 0.040s
- Best performing model: QuadraticDiscriminantAnalysis()
- Score: train: 0.9854, test: 0.9837

In [ ]:
models.append((QuadraticDiscriminantAnalysis(), {}))

---
## Fit and evaluate all models in `models`
Now actually evaluate the models

In [ ]:
clfs = []
for model, param_grid in models:
    print(f'\nTraining {model}')
    start = time()
    clf = RandomizedSearchCV(
        model, param_grid, n_iter=10
    )
    clf = clf.fit(X_train_pca, y_train)
    print(f'- Time taken: {time() - start:.3f}s\n- Best performing model\n`{clf.best_estimator_}`\n- Score: train: {clf.best_score_:.4f}, test: {clf.score(X_test_pca, y_test):.4f}')
    clfs.append(clf.best_estimator_)

### Get detailed analyses of the trained models

In [ ]:
for clf in clfs:
    y_pred = clf.predict(X_test_pca)
    clf_name = f'{str(type(clf))}'.split('.')[-1][:-2]

    print(f"Test set results for {clf_name}")
    print(classification_report(y_test, y_pred, target_names=gesture_to_idx.keys()))

    fig, ax = plt.subplots(figsize=(12,12))
    ConfusionMatrixDisplay.from_estimator(
        clf, 
        X_test_pca, 
        y_test, 
        display_labels=gesture_to_idx.keys(), 
        xticks_rotation="vertical",
        ax=ax,
    )
    plt.title(f'Confusion Matrix of \n{clf}')
    plt.tight_layout()

    plt.savefig(f'imgs/conf_mat_{clf}.pdf')

## Plot the incorrect observations

In [ ]:
clf = clfs[0]
y_pred = clf.predict(X_test_pca)

X_test_incorrect = X_test[y_pred != y_test]
y_pred_incorrect = y_pred[y_pred != y_test]
y_test_incorrect = y_test[y_pred != y_test]
paths_test_incorrect = paths_test[y_pred != y_test]


@interact(idx=(0, y_pred_incorrect.shape[0], 1))
def plot_incorrect(idx=0):
    predicted = idx_to_gesture[y_pred_incorrect[idx]]
    pred_desc = gesture_info[predicted]['description']
    
    actual = idx_to_gesture[y_test_incorrect[idx]]
    actu_desc = gesture_info[actual]['description']
    
    path = '/'.join(paths_test_incorrect[idx].split('/')[3:])
    
    # Create 3 vertical axs:
    # - top is an example of the actual gesture, 
    # - middle is the incorrectly predicted gesture,
    # - bottom is an example of the predicted gesture
    fig, axs = plt.subplots(3)
    
    # First plot an example of the actual gesture
    actual_idx = next(i for i, yi in enumerate(y_train) if yi == y_pred_incorrect[idx])
    gesture_label = idx_to_gesture[y_train[actual_idx]]
    gesture_description = gesture_info[idx_to_gesture[y_train[actual_idx]]]["description"]
    plot_raw_gesture(
        X_train[actual_idx], 
        f'Example of {gesture_label} ({gesture_description})',
        ax=axs[0],
    )

    # Second plot the misclassified gesture
    plot_raw_gesture(
        X_test_incorrect[idx], 
        f'Predicted: {predicted} ({pred_desc})\nActual: {actual} ({actu_desc})',
        ax=axs[1],
    )
    
    # Last plot an example of the predicted gesture
    predicted_idx = next(i for i, yi in enumerate(y_train) if yi == y_test_incorrect[idx])
    gesture_label = idx_to_gesture[y_train[predicted_idx]]
    gesture_description = gesture_info[idx_to_gesture[y_train[predicted_idx]]]["description"]
    plot_raw_gesture(
        X_train[predicted_idx], 
        f'Example of {gesture_label} ({gesture_description})',
        ax=axs[2],
    )
    
    # Finally, tell matplotlib to recompute the layout
    plt.tight_layout()

## Plot 2-component PCA to assess separation

In [ ]:
# PCA can be either 2D or 3D
PLOT_2D = True

# Transform the data via PCA. Either 2 or 3 components are used
pca = PCA(n_components=(2 if PLOT_2D else 3))
X_r = pca.fit(X).transform(X)

# Each observation gets a different colour on the scatter plot, and
# similar colours get different markers to better differentiate them
colours = cm.get_cmap('turbo', n_classes)
markers = ['.', 'x', '*', 'd']


if PLOT_2D:
    # Use 2D subplots
    fig, ax = plt.subplots(figsize=(12,8))
else:
    # Use 3D subplots
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

# Optionally also plot the observation indices along with the points.
# This helps when removing outliers, but increases the amount of clutter
# if PLOT_2D:
#     for i, yi in enumerate(y):
#         ax.annotate(
#             i, 
#             (X_r[i, 0], X_r[i, 1]),
#             color=colours(yi/n_classes),
#             size=5,
#             alpha=0.5,
#         )

# Iterate over each label/gesture
for i, label_idx in enumerate(idx_to_gesture.keys()):
    # Args either has 2 items (if 2D) or 3 (if 3D)
    args = [
        X_r[y == label_idx, 0], 
        X_r[y == label_idx, 1],
    ]
    if not PLOT_2D:
        args.append(X_r[y == label_idx, 2])
    
    # Get a shortened version of the gesture index for the legend
    gesture_idx = idx_to_gesture[label_idx].replace('gesture', '')
    # Get the short gesture description for the legend
    gesture_desc = gesture_info[idx_to_gesture[label_idx]]["desc"]
    
    # Actually plot the points, either in 2 or 3 dimensions
    ax.scatter(
        *args,
        color=colours(label_idx/n_classes),
        alpha=0.3,
        s=10,
        marker=markers[label_idx % 4],
        label=f'{gesture_idx} ({gesture_desc})'
    )

# ----------------------------------------------------------------
#
#   modified from https://stackoverflow.com/a/4701285/14555505
#
# Shrink current axis's height by 10% on the bottom so the legend will fit
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.2,
                 box.width, box.height * 0.80])
# Put a legend below current axis in the newly made space
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=4)
# ----------------------------------------------------------------

# Give the plot a title and save it
plt.title(f"PCA with {'two' if PLOT_2D else 'three'} components over {n_classes} gestures")
plt.savefig(f'imgs/{2 if PLOT_2D else 3}_pca_{n_classes}_classes_{n_obs}_obs.pdf')

Define a widget that, given an observation's index, will display the raw sensor measurements. This is
useful for identifying and removing outliers or bad observations.

In [ ]:
@interact(idx='0')
def plot_from_index(idx='0'):
    if len(idx) == 0:
        return
    idx = int(idx)
    gesture_idx = idx_to_gesture[y[idx]]
    plot_raw_gesture(
        X[idx], 
        f'{gesture_idx}: {gesture_info[gesture_idx]["description"]}\n{paths[idx]}'
    )
    print(f'{gesture_info[gesture_idx]["description"]}')
    print('rm ' + '/'.join(paths[idx].split('/')[3:]))

## Train TensorFlow neural network

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
if USE_W_AND_B:
    import wandb
    from wandb.keras import WandbCallback

In [ ]:
config = {
    "epochs": 10,
    "batch_size": 32,
    "resample_period": 25,
}
if USE_W_AND_B:
    wandb.init(project="ergo", entity="beyarkay")
    wandb.config = config

In [ ]:
# https://www.tensorflow.org/tutorials/load_data/csv#multiple_files_2
dirs = sorted(list(dir_files.keys()))

list_ds = tf.data.Dataset.list_files('../gesture_data/train/*/*.txt')

def preprocess_features(numbers):
    # Convert the np.array to a pd.DataFrame
    df = pd.DataFrame(data=numbers.numpy())
    
    # Set the index to be column 0 (The column containing the miliseconds 
    # since the start of the gesture)
    df.index = pd.TimedeltaIndex(df[0], unit='ms', name='offset_ms')
    
    # Delete the milliseconds column (We won't use it for training)
    del df[0]
    
    # If the start and end items don't explicitly exist => add them
    start = pd.Timedelta('0 days 00:00:00.000')
    end = pd.Timedelta('0 days 00:00:00.975')
    if start not in df.index:
        df.loc[start] = pd.Series(dtype='float64')
    if end not in df.index:
        df.loc[end] = pd.Series(dtype='float64')

    # Resample the data so we've got values exactly every 25ms
    df = df.resample(f"{config['resample_period']}ms").mean().ffill()
    
    # Normalise the data to have zero-mean and unit-variance
    df = (df - df.stack().mean()) / df.stack().std()
    return np.array(df)
    
    
def preprocess_label(label):
#     print(label)
    label //=  2
    return tf.keras.utils.to_categorical(label-1, num_classes=len(dirs))


@tf.function
def process_path(file_path):
    # Get the label of the observation from the file path
    label = tf.strings.split(file_path, os.sep)[-2]    
    label = tf.strings.regex_replace(label, "gesture", "")
    label = tf.strings.to_number(label, tf.int32)
    [label,] = tf.py_function(preprocess_label, [label], [tf.float32])
    label.set_shape(len(dirs))
    
    # Read in the actual file
    file = tf.io.read_file(file_path)
    # Split by newlines to get an array of each line
    lines = tf.strings.split(file, "\n")
    # Split each line by ',' to get an array of arrays of strings
    items = tf.strings.split(lines, ',')
    # Convert the array of array of strings to a 2D array of float32
    nums = tf.strings.to_number(items, out_type=tf.dtypes.float32)
    # preprocess the raw sensor values via pandas
    [nums,] = tf.py_function(preprocess_features, [nums], [tf.float32])
    nums.set_shape((40, 30))
    
    return nums, label

labeled_ds = list_ds.map(process_path)
labeled_ds = labeled_ds.shuffle(buffer_size=1000)
labeled_ds = labeled_ds.batch(config['batch_size'])

for sensor_data, label in labeled_ds.take(1):
    print(list(zip(sensor_data.numpy(), label.numpy()))[0])


In [ ]:
model = tf.keras.Sequential([
    keras.layers.Flatten(input_shape=(40, 30), name='input'),
    keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(len(dirs), name='output'),
], name='Ergo')

model.summary()

metrics = [
#     tf.keras.metrics.Accuracy(),
#     tf.keras.metrics.Precision(),
#     tf.keras.metrics.Recall(),
#     tf.keras.metrics.TrueNegatives(),
#     tf.keras.metrics.TruePositives(),
#     tf.keras.metrics.FalseNegatives(),
#     tf.keras.metrics.FalsePositives(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.CategoricalCrossentropy(),
#     tf.keras.metrics.MeanAbsolutePercentageError(),
]

model.compile(
    optimizer='adam',
    # Use `categorical_crossentropy` because the labels are one-hot-encoded
    loss='categorical_crossentropy',
    metrics=metrics,
)

callbacks = [] if not USE_W_AND_B else [WandbCallback()]

history = model.fit(
    labeled_ds, 
    epochs=config['epochs'],
    callbacks=callbacks
)

# # You can also evaluate or predict on a dataset.
# print("Evaluate")
# result = model.evaluate(labeled_ds)
# dict(zip(model.metrics_names, result))


In [ ]:
predictions = model.predict(labeled_ds)
predictions[0]

In [ ]:
i = 0
for d, l in labeled_ds.take(5):
    for item in l:
        print(np.argmax(predictions[i]), np.argmax(item))
        i += 1
        